In [1]:
from IPython.display import JSON

import altair as alt
# Uncomment/run this line to enable Altair in JupyterLab/nteract:
alt.enable_mime_rendering()

import ipywidgets as widgets
from ipywidgets import fixed, interact

from openfisca_core import decompositions, periods
from openfisca_france import FranceTaxBenefitSystem
from openfisca_matplotlib import graphs, utils

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tbs = FranceTaxBenefitSystem()

## Display "Revenu disponible" interactively

In [3]:
def calculate_revenu_disponible(age, salaire_de_base, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
            "salaire_de_base": salaire_de_base, # Annual basis
        },
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    revenu_disponible = simulation.calculate("revenu_disponible", period)
    return revenu_disponible

Try with an input text:

In [4]:
interact(
    calculate_revenu_disponible,
    salaire_de_base=widgets.BoundedFloatText(min=0, max=150000, step=100, value=15000, description="Salaire de base"),
    age=fixed(30),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(BoundedFloatText(value=15000.0, description=u'Salaire de base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Note: press Enter after you type a new value to recompute.

Try with two sliders, for "age" and "salaire_de_base":

In [5]:
interact(
    calculate_revenu_disponible,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    salaire_de_base=widgets.FloatSlider(min=0, max=150000, step=100, value=15000, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), FloatSlider(value=15000.0, continuous_update=False, description=u'salaire_de_base', max=150000.0, step=100.0), Output()), _dom_classes=('widget-interact',))

Note: you can edit the number at the right of the slider. Press Enter after you type a new value to recompute.

This is quite slow to recompute and redraw if we don't use `continuous_update=False`.
This is mainly because we create a new simulation each time, in `calculate_revenu_disponible`. I did not find how to update a variable of an existing simulation, I'm not sure it is possible with OpenFisca.

Another approach would be to pre-calculate all the values for a range of "Salaires", with a step.

In [6]:
def precalculate_revenu_disponible(period, min, max, count):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": 30,
        },
        "axes": [
            dict(
                count = count,
                min = min,
                max = max,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    revenu_disponible = simulation.calculate("revenu_disponible", period)
    return revenu_disponible

In [7]:
def count_to_step(min, max, count):
    """Examples:
    >>> count_to_step(0, 80, 5)
    20
    """
    return (max - min) / (count - 1)

def value_to_index(min, step, value):
    """Examples:
    >>> value_to_index(0, 10, 0)
    0
    >>> value_to_index(0, 10, 40)
    4
    >>> value_to_index(3, 1, 6)
    3
    """
    return int((value / step) - min)

In [8]:
count = 12
min = 0
max = 50000
step = count_to_step(min, max, count)
initial_value = 3 * step

In [9]:
revenu_disponible = precalculate_revenu_disponible(periods.period("2015"), min, max, count)
revenu_disponible

array([  4791.49023438,   7324.55273438,   9857.53808594,  12390.6328125 ,
        14923.69726562,  17176.4609375 ,  19909.40234375,  22918.22851562,
        25998.22070312,  28644.80078125,  31254.34179688,  33863.57421875], dtype=float32)

In [10]:
def display_revenu_disponible(salaire_de_base, revenu_disponible, min, step):
    index = value_to_index(min, step, salaire_de_base)
    return revenu_disponible[index]

In [11]:
interact(
    display_revenu_disponible,
    salaire_de_base=widgets.FloatSlider(min=min, max=max, step=step, value=initial_value),
    revenu_disponible=fixed(revenu_disponible),
    min=fixed(min),
    step=fixed(step),
)
None  # Hide strange output

/home/cbenz/.virtualenvs/openfisca-interactive-TZbnw1I8/lib/python2.7/site-packages/traitlets/traitlets.py:567: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  silent = bool(old_value == new_value)


interactive(children=(FloatSlider(value=13635.0, description=u'salaire_de_base', max=50000.0, step=4545.0), Output()), _dom_classes=('widget-interact',))

## Display waterfall

In [12]:
def display_waterfall(age, salaire_de_base, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
            "salaire_de_base": salaire_de_base, # Annual basis
        },
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    graphs.draw_waterfall(simulation)

In [13]:
interact(
    display_waterfall,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    salaire_de_base=widgets.FloatSlider(min=min, max=max, step=step, value=initial_value, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), FloatSlider(value=13635.0, continuous_update=False, description=u'salaire_de_base', max=50000.0, step=4545.0), Output()), _dom_classes=('widget-interact',))

Here again, another approach is to pre-calculate results.

In [14]:
def precalculate_waterfall(min, max, count, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": 30,
        },
        "axes": [
            dict(
                count = count,
                min = min,
                max = max,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    decomposition_json = decompositions.get_decomposition_json(tbs)
    decomposition_json_precalculated = decompositions.calculate([simulation], decomposition_json)
    return decomposition_json_precalculated

In [15]:
decomposition_json_precalculated = precalculate_waterfall(min, max, count, periods.period("2015"))
JSON(decomposition_json_precalculated)

<IPython.core.display.JSON object>

In [16]:
def update_key(mapping, key, value):
    return {
        k: value if k == key else v
        for k, v in mapping.iteritems()
    }

def keep_value_at_index(index, node):
    """Transform decomposition JSON tree, keeping only the given index for each node["values"]."""
    if "values" in node:
        node = update_key(node, "values", [node["values"][index]])
    if "children" in node:
        new_children = [
            keep_value_at_index(index, child_node)
            for child_node in node["children"]
        ]
        node = update_key(node, "children", new_children)
    return node

In [17]:
JSON(keep_value_at_index(0, decomposition_json_precalculated))

<IPython.core.display.JSON object>

In [18]:
def display_precalculated_waterfall(salaire_de_base, min, step):
    index = value_to_index(min, step, salaire_de_base)
    out_node = utils.OutNode()
    utils.convert_to_out_node(out_node, keep_value_at_index(index, decomposition_json_precalculated))
    out_node.setLeavesVisible()
    fig = plt.figure()
    axes = fig.gca()
    graphs.draw_waterfall_from_node_data(out_node, axes, tbs.CURRENCY)

In [19]:
interact(
    display_precalculated_waterfall,
    salaire_de_base=widgets.FloatSlider(min=min, max=max, step=step, value=initial_value),
    min=fixed(min),
    step=fixed(step),
)
None  # Hide strange output

interactive(children=(FloatSlider(value=13635.0, description=u'salaire_de_base', max=50000.0, step=4545.0), Output()), _dom_classes=('widget-interact',))

## Display bareme

Instead of varying the salary with a slider, let's climb on the [ladder of abstraction](http://worrydream.com/LadderOfAbstraction/) to see *all* the possible waterfall charts at a time. This new chart is called a "bareme" in French.

In [22]:
def display_bareme(age, period):
    scenario_params = {
        "period": period,
        "parent1": {
            "age": age,
        },
        "axes": [
            dict(
                count = 10,
                min = 0,
                max = 50000,
                name = 'salaire_de_base',
            ),
        ],
    }
    scenario = tbs.new_scenario().init_single_entity(**scenario_params)
    simulation = scenario.new_simulation()
    graphs.draw_bareme(
        simulation = simulation,
        x_axis = "salaire_imposable",
        legend_position = 2,
        bbox_to_anchor = (1.05, 1),
    )

In [23]:
interact(
    display_bareme,
    age=widgets.IntSlider(min=0, max=130, step=1, value=30, continuous_update=False),
    period=fixed(periods.period("2015")),
)
None  # Hide strange output

interactive(children=(IntSlider(value=30, continuous_update=False, description=u'age', max=130), Output()), _dom_classes=('widget-interact',))